# Simulation setup

In [1]:
# libs
import sys, logging
sys.path.append("/home/rado/Projects/wealthManagement/plutus/")

from context.instruments import CurrentAccount, Mortgage
from context.assets import RealEstate
from context.agreements import RentalAgreement, UtilitiesAgreement, EmployeeContract
from context.party import Party
from core.plots import PlutusPlotter

# delete log
import os
if os.path.exists("plutus.log"):
  os.remove("plutus.log")

## [1] General

In [2]:
config = {}
config['n_years'] = 1

config['logLevel'] = logging.INFO
config['logPath'] = 'plutus.log'

In [3]:
# loggers
logger = logging.getLogger('plutusLogger')
logger.setLevel(config['logLevel'])
formatter = logging.Formatter('%(asctime)s :: %(name)s | %(levelname)s | %(message)s')

fileHandler = logging.FileHandler(config['logPath'])
fileHandler.setFormatter(formatter)
logger.addHandler(fileHandler)

streamHandler = logging.StreamHandler()
streamHandler.setFormatter(formatter)
logger.addHandler(streamHandler)

In [4]:
logger.info('============================================')
logger.info('============ CONFIGURATION  ================')
logger.info('============================================')
logger.info('')
logger.info('[1] General: ')
logger.info(' - n_years: '+str(config['n_years']))

2020-11-21 20:42:13,462 :: plutusLogger | INFO | ============================================
2020-11-21 20:42:13,503 :: plutusLogger | INFO | ============ CONFIGURATION  ================
2020-11-21 20:42:13,504 :: plutusLogger | INFO | ============================================
2020-11-21 20:42:13,505 :: plutusLogger | INFO | 
2020-11-21 20:42:13,506 :: plutusLogger | INFO | [1] General: 
2020-11-21 20:42:13,507 :: plutusLogger | INFO |  - n_years: 1


## [2] Market

In [5]:
logger.info('')
logger.info('[2] Market: ')
# inflation_index
logger.info('Adding inflation index.')
inflation_index={}
inflation_rate=0.03
inflation_index[0]=1.0
for step in range(1,config['n_years']*12+1):
    inflation_index[step]=inflation_index[step-1]+inflation_rate/12
    logger.debug('Inflation index step ' + str(step) + ' : ' + str(inflation_index[step]))

logger.info('Adding real-estate index.')
# real-estate market
re_index={}
re_rate=0.07
re_index[0]=1.0
for step in range(1,config['n_years']*12+1):
    re_index[step]=re_index[step-1]+re_rate/12
    logger.debug('Real estate index step ' + str(step) + ' : ' + str(re_index[step]))

2020-11-21 20:42:14,461 :: plutusLogger | INFO | 
2020-11-21 20:42:14,463 :: plutusLogger | INFO | [2] Market: 
2020-11-21 20:42:14,464 :: plutusLogger | INFO | Adding inflation index.
2020-11-21 20:42:14,465 :: plutusLogger | INFO | Adding real-estate index.


## [3] Portfolio

In [6]:
logger.info('')
logger.info('[3] Portfolio: ')
current_account = CurrentAccount(id='beznyUcet'
                                 , current_outstanding=1e5
                                 , monthly_cost= 100
                                 , cnit=0.0
                                 , logger=logger)

mortgage = Mortgage(id='hypoteka'
                    , principal=3e6
                    , cnit=0.02
                    , maturity_in_years=30
                    , logger=logger)

flat = RealEstate(id='byt'
                  , current_market_value=5e6
                  , property_tax = 2000
                  , renovation_costs = 2e5
                  , renovation_steps = {5, 10}
                  , house_community_costs = 6000
                  , real_estate_index = re_index
                  , logger=logger)

employeeContract = EmployeeContract(id='zamestnanec_Credo'
                                    , average_yearly_validity = 1.0
                                    , salary = 65000
                                    , bonus_steps= set(range(6,361,6))
                                    , bonus_rate = 0.3
                                    , yearly_salary_trend = 0.05
                                    , income_tax_rate = 0.42
                                    , logger=logger)

2020-11-21 20:42:15,323 :: plutusLogger | INFO | 
2020-11-21 20:42:15,324 :: plutusLogger | INFO | [3] Portfolio: 
2020-11-21 20:42:15,325 :: plutusLogger | INFO | Initializing: beznyUcet
2020-11-21 20:42:15,326 :: plutusLogger | INFO | Initializing: hypoteka
2020-11-21 20:42:15,327 :: plutusLogger | INFO | Initializing: byt
2020-11-21 20:42:15,328 :: plutusLogger | INFO | Initializing: employeeCredo


In [7]:
# --- Party ---
party = Party(name= 'Rado'
             , initial_free_resources = 10000
             , initial_portfolio = {current_account, mortgage, flat, employeeContract}
             , monthly_expenditures = 20000)

TypeError: __init__() missing 2 required positional arguments: 'logger' and 'discount_rate'

In [ ]:
for step in range(0, config['n_years']*12+1):
    party.live(step)

In [ ]:
mortgage.calculateNumberOfPeriods()
mortgage.calculateMonthlyPayment()
paid_principal = 0
paid_interest = 0

print(mortgage.monthly_interest_rate)

for step in range(0,mortgage.number_of_periods+1):
    mortgage.calculateInterestPayment(step)
    mortgage.calculatePrincipalPayment(step)
    mortgage.calculateOutstandingAmount(step)
    
    paid_principal = paid_principal + mortgage.principal_payment[step]
    paid_interest = paid_interest + mortgage.interest_payment[step]
    
    print('[step ' + str(step) + '] outstanding: ' + "%.2f"%mortgage.outstanding_amount[step] +
         ' | mp: ' + "%.2f"%mortgage.monthly_payment +
         ' | pp: ' + "%.2f"%mortgage.principal_payment[step] +
         ' | ip: ' + "%.2f"%mortgage.interest_payment[step] +
         ' | total pp: ' + "%.2f"%paid_principal +
         ' | total ip: ' + "%.2f"%paid_interest)

In [ ]:
inflation = 0.03
market_trend = 0.02

for step in range(0,10):
    flat.projectPrice(step, inflation, market_trend)
    print('[step ' + str(step) + '] flat price: ' + "%.2f"%flat.price[step])

In [ ]:
property_tax= 25
renovation_costs = 200
renovation_steps = {5, 10}
house_community_costs = 15
    
for step in range(0,11):
    flat.projectCosts(step, property_tax, renovation_costs, renovation_steps, house_community_costs)
    print('[step ' + str(step) + '] flat costs: ' + "%.2f"%flat.costs[step])